Задача

1) Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:

1. название источника;
2. наименование новости;
3. ссылку на новость;
4. дата публикации.

2) Сложить все новости в БД


In [174]:
from pprint import pprint
from pymongo import MongoClient
from lxml import html
import pandas as pd
import requests
import time
from datetime import date

In [175]:
# Настраиваем рабочие ссылки и браузер
main_link_ma = 'https://news.mail.ru'
main_link_le = 'https://lenta.ru'
main_link_ya = 'https://yandex.ru'

header = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36 OPR/32.0.1948.25'}

### Mail

In [176]:
def mail_data():
    
    # Получаем запрос
    response = requests.get(main_link_ma, headers=header)
    root = html.fromstring(response.text)
    
    # Ищем ссылки новостей
    links = root.xpath('//td/div/a/@href')
    
    df_mail = pd.DataFrame()
    mail_news = []
    # Переходим по ним и внутри собираем нужную информацию
    for i in range(len(links)):
        # Получаем запрос
        response_li = requests.get(f'{main_link_ma}{links[i]}', headers=header)
        root_li = html.fromstring(response_li.text)
        # Получаем дату, источник, заголовок и упаковываем в датафрейм через словарь
        date = root_li.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")
        source = root_li.xpath("//a[@class='link color_gray breadcrumbs__link']//span[@class='link__text']/text()")
        news =  root_li.xpath("//h1[@class='hdr__inner']/text()")
        mail_new = {
            'source': source[0],
            'news': news[0],
            'link': main_link_ma+links[i],
            'date': date[0][:10]
        }
        mail_news.append(mail_new)
    df_mail = pd.DataFrame(mail_news)
    return df_mail

### Lenta.ru

In [177]:
def lenta_data():
    
    # Получаем запрос
    response_le = requests.get(main_link_le, headers=header)
    root_le = html.fromstring(response_le.text)

    # Ищем данные
    # Ссылки
    links = root_le.xpath("//div[contains(@class,'span4')]/div[contains(@class,'item')]/a/@href")
    links = [f'{main_link_le}{link}' for link in links]
    for i in range(len(links)):
        if 'ruhttps' in links[i]:
            links[i] = links[i][16:] #обрабатываем случаи с полной ссылкой 
    # Дата
    date_1 = root_le.xpath("//div[contains(@class,'span4')]/div[contains(@class,'item')]/h2/a/time/@title")
    date = date_1+root_le.xpath("//div[contains(@class,'span4')]/div[contains(@class,'item')]/a//time/@title")
    # Источник
    source = ['lenta.ru' for i in range(len(links))]
    # Текст новости
    news_1 = root_le.xpath("//div[contains(@class,'span4')]/div[contains(@class,'item')]//h2/a/text()")
    news =  news_1+root_le.xpath("//div[contains(@class,'span4')]/div[contains(@class,'item')]/a/text()")
    news = [new.replace('\xa0', ' ') for new in news]

    # Складываем в датафрейм
    df_lenta = pd.DataFrame()
    lenta_news = []
    for i in range(len(links)):
        lenta_new = {
                'source': source[i],
                'news': news[i],
                'link': links[i],
                'date': date[i]
            }
        lenta_news.append(lenta_new)
    df_lenta = pd.DataFrame(lenta_news)
    return df_lenta 

### Yandex

In [178]:
def yandex_data():
    # Получаем запрос
    response_ya = requests.get(f'{main_link_ya}/news/', headers=header)
    root_ya = html.fromstring(response_ya.text)

    # Ищем данные
    # Ссылки
    links = root_ya.xpath("//div[@class='stories-set stories-set_main_yes stories-set_pos_0']//a[contains(@class, 'link_theme_black')]/@href")
    links = [f'{main_link_ya}{link}' for link in links]
    # Дата
    date_ya = root_ya.xpath("//div[@class='stories-set stories-set_main_yes stories-set_pos_0']//div[@class='story__date']/text()")
    for i in range(len(date_ya)):
        if ':' in date_ya[i]:
            date_ya[i] = f'{date.today()}'
        else:
            date_ya[i] = date_ya[i][-10:] #обработка даты
    # Источник
    source = root_ya.xpath("//div[@class='stories-set stories-set_main_yes stories-set_pos_0']//div[@class='story__date']/text()")
    for i in range(len(source)):
        if ':' in source[i]:
            source[i] = source[i][:-6]
        else:
            source[i] = source[i][:-10] #обработка источника
    # Текст новости
    news =  root_ya.xpath("//div[@class='stories-set stories-set_main_yes stories-set_pos_0']//h2[@class='story__title']/a/text()")

    # Складываем в датафрейм
    df_yandex = pd.DataFrame()
    yandex_news = []
    for i in range(len(links)):
        yandex_new = {
                    'source': source[i],
                    'news': news[i],
                    'link': links[i],
                    'date': date_ya[i]
                }
        yandex_news.append(yandex_new)
    df_yandex = pd.DataFrame(yandex_news)
    return df_yandex

In [179]:
df=pd.concat([mail_data(),lenta_data(),yandex_data()], ignore_index=True, axis=0)
df

,source,news,link,date
0,Коммерсантъ,От Михаила Ефремова не ждут компенсации,https://news.mail.ru/incident/42143094/,2020-06-10
1,Новости Mail.ru,Коронавирус COVID-19: главные события 10 июня,https://news.mail.ru/society/42140634/,2020-06-10
2,Интерфакс,Низкая смертность среди заразившихся COVID-19 ...,https://news.mail.ru/society/42142113/,2020-06-10
3,ТАСС Наука,На рынках Вьетнама нашли «скрытые резервуары» ...,https://news.mail.ru/society/42138289/,2020-06-10
4,Известия,Прерванный смех: умер сатирик Анатолий Трушкин,https://news.mail.ru/society/42136536/,2020-06-10
5,lenta.ru,В рамках расследования ДТП с Ефремовым возбуди...,https://lenta.ru/news/2020/06/10/narkos/,10 июня 2020
6,lenta.ru,В США предложили признать Россию «спонсором те...,https://lenta.ru/news/2020/06/10/hardsanc/,10 июня 2020
7,lenta.ru,В России задумались о пожизненном лишении прав...,https://lenta.ru/news/2020/06/10/prava/,10 июня 2020
8,lenta.ru,50-летняя актриса показала тело в бикини и обр...,https://lenta.ru/news/2020/06/10/50model/,10 июня 2020
9,lenta.ru,Восстановлена хронология походов Ефремова в ба...,https://lenta.ru/news/2020/06/10/bars/,10 июня 2020


In [180]:
client = MongoClient('localhost',27017)

db = client['news']

# Создаём коллекции 
collection_news = db.collection_news

# Добавляем данные в коллекции, преобразуя их в словарь
collection_news.insert_many(df.to_dict('records'))

In [181]:
# Проверка
collection_news.find().count()

c:\users\игорь\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


20

In [182]:
#Вывод содержимого БД
for post in collection_news.find():
    pprint(post)

{'_id': ObjectId('5ee13d13557e610537631949'),
 'date': '2020-06-10',
 'link': 'https://news.mail.ru/incident/42143094/',
 'news': 'От Михаила Ефремова не ждут компенсации',
 'source': 'Коммерсантъ'}
{'_id': ObjectId('5ee13d13557e61053763194a'),
 'date': '2020-06-10',
 'link': 'https://news.mail.ru/society/42140634/',
 'news': 'Коронавирус COVID-19: главные события 10 июня',
 'source': 'Новости Mail.ru'}
{'_id': ObjectId('5ee13d13557e61053763194b'),
 'date': '2020-06-10',
 'link': 'https://news.mail.ru/society/42142113/',
 'news': 'Низкая смертность среди заразившихся COVID-19 в России смутила '
         'представителя ВОЗ',
 'source': 'Интерфакс'}
{'_id': ObjectId('5ee13d13557e61053763194c'),
 'date': '2020-06-10',
 'link': 'https://news.mail.ru/society/42138289/',
 'news': 'На рынках Вьетнама нашли «скрытые резервуары» коронавирусов',
 'source': 'ТАСС Наука'}
{'_id': ObjectId('5ee13d13557e61053763194d'),
 'date': '2020-06-10',
 'link': 'https://news.mail.ru/society/42136536/',
 'news'

In [183]:
collection_news.drop({})